## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page [Extract clinical events and relations](https://aws.amazon.com/marketplace/pp/prodview-oi26faao26z54)
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline to find clinical events and find temporal relations (ERA)

Overview:

Clinical entity detection, assertion status assignment, and relation extraction are crucial for analyzing medical texts. These processes can be incredibly useful to healthcare professionals, researchers, and medical NLP practitioners, enabling the extraction of valuable insights from clinical literature, electronic health records, and patient notes.


- **Model**: `en.explain_doc.era`

- **Model Description**: A pipeline with `ner_clinical_events`, `assertion_dl` and `re_temporal_events_clinical`. It will extract clinical entities, assign assertion status and find temporal relationships between clinical entities.


In [1]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [2]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/base_serializers.py:28: UserWarning: A NumPy version >=1.23.5 and <2.3.0 is required for this version of SciPy (detected version 1.22.4)
  import scipy.sparse


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

In [4]:
model_name = "en-explain-doc-era"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.2xlarge"

## 2. Create a deployable model from the model package.

In [5]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, 
    model_package_arn=model_package_arn,
    sagemaker_session=sagemaker_session,
)

### Input Format

To use the model, you need to provide input in one of the following supported formats:

#### JSON Format

Provide input as JSON. We support two variations within this format:

1. **Array of Text Documents**: 
   Use an array containing multiple text documents. Each element represents a separate text document.

   ```json
   {
       "text": [
           "Text document 1",
           "Text document 2",
           ...
       ]
   }

    ```

2. **Single Text Document**:
   Provide a single text document as a string.


   ```json
    {
        "text": "Single text document"
    }
   ```

#### JSON Lines (JSONL) Format

Provide input in JSON Lines format, where each line is a JSON object representing a text document.

```
{"text": "Text document 1"}
{"text": "Text document 2"}
```

## 3. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

### A. Deploy the SageMaker model to an endpoint

In [6]:
# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=real_time_inference_instance_type, 
    endpoint_name=model_name,
)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [7]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [8]:
docs = [
    '''The patient is a 21-day-old Caucasian male here for 2 days of congestion - mom has been suctioning yellow discharge from the patient's nares, plus she has noticed some mild problems with his breathing while feeding (but negative for any perioral cyanosis or retractions). One day ago, mom also noticed a tactile temperature and gave the patient Tylenol. Baby also has had some decreased p.o. intake. His normal breast-feeding is down from 20 minutes q.2h. to 5 to 10 minutes secondary to his respiratory congestion. He sleeps well, but has been more tired and has been fussy over the past 2 days. The parents noticed no improvement with albuterol treatments given in the ER. His urine output has also decreased; normally he has 8 to 10 wet and 5 dirty diapers per 24 hours, now he has down to 4 wet diapers per 24 hours. Mom denies any diarrhea. His bowel movements are yellow colored and soft in nature.''',
    '''The patient is a 40-year-old white male who presents with a chief complaint of "chest pain". The patient is diabetic and has a prior history of coronary artery disease. The patient presents today stating that his chest pain started yesterday evening and has been somewhat intermittent. He has been advised Aspirin 81 milligrams QDay. Humulin N. insulin 50 units in a.m. HCTZ 50 mg QDay. Nitroglycerin 1/150 sublingually PRN chest pain.'''
]


sample_text = """She is admitted to The John Hopkins Hospital 2 days ago with a history of gestational diabetes mellitus diagnosed. She denied pain and any headache. She was seen by the endocrinology service and she was discharged on 03/02/2018 on 40 units of insulin glargine, 12 units of insulin lispro, and metformin 1000 mg two times a day. She had close follow-up with endocrinology post discharge. """


### JSON

In [9]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
ner_df = pd.DataFrame(data[0]["ner_predictions"])
assertion_df = pd.DataFrame(data[0]["assertion_predictions"])
relation_df = pd.DataFrame(data[0]["relation_predictions"])

In [10]:
ner_df

,ner_chunk,begin,end,ner_label,ner_confidence
0,admitted,7,14,OCCURRENCE,0.9958
1,The John Hopkins Hospital,19,43,CLINICAL_DEPT,0.859
2,2 days ago,45,54,DATE,0.80329996
3,gestational diabetes mellitus,74,102,PROBLEM,0.8622667
4,denied,119,124,EVIDENTIAL,0.9585
5,pain,126,129,PROBLEM,0.9996
6,any headache,135,146,PROBLEM,0.93675
7,the endocrinology service,165,189,CLINICAL_DEPT,0.58563334
8,discharged,203,212,OCCURRENCE,0.9679
9,03/02/2018,217,226,DATE,0.9979


In [11]:
assertion_df

,ner_chunk,begin,end,ner_label,assertion,assertion_confidence
0,admitted,7,14,OCCURRENCE,present,0.9991
1,The John Hopkins Hospital,19,43,CLINICAL_DEPT,present,0.9999
2,2 days ago,45,54,DATE,present,0.9993
3,gestational diabetes mellitus,74,102,PROBLEM,present,0.9772
4,denied,119,124,EVIDENTIAL,absent,0.9886
5,pain,126,129,PROBLEM,absent,0.9992
6,any headache,135,146,PROBLEM,absent,1.0
7,the endocrinology service,165,189,CLINICAL_DEPT,present,0.9995
8,discharged,203,212,OCCURRENCE,present,0.9995
9,03/02/2018,217,226,DATE,present,0.9995


In [12]:
relation_df

,ner_chunk1,ner_chunk1_begin,ner_chunk1_end,ner_label1,ner_chunk2,ner_chunk2_begin,ner_chunk2_end,ner_label2,relations,relation_confidence
0,admitted,7,14,OCCURRENCE,The John Hopkins Hospital,19,43,CLINICAL_DEPT,AFTER,0.9621104
1,admitted,7,14,OCCURRENCE,2 days ago,45,54,DATE,OVERLAP,0.99970835
2,admitted,7,14,OCCURRENCE,gestational diabetes mellitus,74,102,PROBLEM,BEFORE,0.99985516
3,The John Hopkins Hospital,19,43,CLINICAL_DEPT,2 days ago,45,54,DATE,OVERLAP,0.85771245
4,The John Hopkins Hospital,19,43,CLINICAL_DEPT,gestational diabetes mellitus,74,102,PROBLEM,BEFORE,0.90553373
5,2 days ago,45,54,DATE,gestational diabetes mellitus,74,102,PROBLEM,OVERLAP,0.91259193
6,denied,119,124,EVIDENTIAL,pain,126,129,PROBLEM,BEFORE,0.9999995
7,denied,119,124,EVIDENTIAL,any headache,135,146,PROBLEM,BEFORE,1.0
8,pain,126,129,PROBLEM,any headache,135,146,PROBLEM,OVERLAP,1.0
9,the endocrinology service,165,189,CLINICAL_DEPT,discharged,203,212,OCCURRENCE,OVERLAP,0.58700085


### JSON Lines

In [13]:
import json

def create_jsonl(records):
    
    if isinstance(records, str):
        records = [records]
    
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

In [14]:
input_jsonl_data = create_jsonl(sample_text)
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"ner_predictions": [{"ner_chunk": "admitted", "begin": 7, "end": 14, "ner_label": "OCCURRENCE", "ner_confidence": "0.9958"}, {"ner_chunk": "The John Hopkins Hospital", "begin": 19, "end": 43, "ner_label": "CLINICAL_DEPT", "ner_confidence": "0.859"}, {"ner_chunk": "2 days ago", "begin": 45, "end": 54, "ner_label": "DATE", "ner_confidence": "0.80329996"}, {"ner_chunk": "gestational diabetes mellitus", "begin": 74, "end": 102, "ner_label": "PROBLEM", "ner_confidence": "0.8622667"}, {"ner_chunk": "denied", "begin": 119, "end": 124, "ner_label": "EVIDENTIAL", "ner_confidence": "0.9585"}, {"ner_chunk": "pain", "begin": 126, "end": 129, "ner_label": "PROBLEM", "ner_confidence": "0.9996"}, {"ner_chunk": "any headache", "begin": 135, "end": 146, "ner_label": "PROBLEM", "ner_confidence": "0.93675"}, {"ner_chunk": "the endocrinology service", "begin": 165, "end": 189, "ner_label": "CLINICAL_DEPT", "ner_confidence": "0.58563334"}, {"ner_chunk": "discharged", "begin": 203, "end": 212, "ner_label":

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 4. Batch inference

In [16]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [17]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    # Define a dictionary with DataFrame objects and their names
    display(data)
    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [20]:
process_s3_json_output_and_save(validation_json_file_name)

[{'ner_predictions': [{'ner_chunk': '2 days',
    'begin': 52,
    'end': 57,
    'ner_label': 'DURATION',
    'ner_confidence': '0.97905004'},
   {'ner_chunk': 'congestion',
    'begin': 62,
    'end': 71,
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.9915'},
   {'ner_chunk': 'suctioning yellow discharge',
    'begin': 88,
    'end': 114,
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.5022667'},
   {'ner_chunk': 'some mild problems with his breathing',
    'begin': 163,
    'end': 199,
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.6756833'},
   {'ner_chunk': 'feeding',
    'begin': 207,
    'end': 213,
    'ner_label': 'OCCURRENCE',
    'ner_confidence': '0.4077'},
   {'ner_chunk': 'any perioral cyanosis',
    'begin': 233,
    'end': 253,
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.7358'},
   {'ner_chunk': 'retractions',
    'begin': 258,
    'end': 268,
    'ner_label': 'PROBLEM',
    'ner_confidence': '0.9899'},
   {'ner_chunk': 'One day ago',
    'begi

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [23]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"ner_predictions": [{"ner_chunk": "2 days", "begin": 52, "end": 57, "ner_label": "DURATION", "ner_confidence": "0.97905004"}, {"ner_chunk": "congestion", "begin": 62, "end": 71, "ner_label": "PROBLEM", "ner_confidence": "0.9915"}, {"ner_chunk": "suctioning yellow discharge", "begin": 88, "end": 114, "ner_label": "PROBLEM", "ner_confidence": "0.5022667"}, {"ner_chunk": "some mild problems with his breathing", "begin": 163, "end": 199, "ner_label": "PROBLEM", "ner_confidence": "0.6756833"}, {"ner_chunk": "feeding", "begin": 207, "end": 213, "ner_label": "OCCURRENCE", "ner_confidence": "0.4077"}, {"ner_chunk": "any perioral cyanosis", "begin": 233, "end": 253, "ner_label": "PROBLEM", "ner_confidence": "0.7358"}, {"ner_chunk": "retractions", "begin": 258, "end": 268, "ner_label": "PROBLEM", "ner_confidence": "0.9899"}, {"ner_chunk": "One day ago", "begin": 272, "end": 282, "ner_label": "DATE", "ner_confidence": "0.80270004"}, {"ner_chunk": "a tactile temperature", "begin": 302, "end": 322

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: en-explain-doc-era-2024-09-06-08-46-39-002


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

